# Global-Wheat-Detection: EfficientDet Inference

This is the inference notebook to my EfficientDet pipeline. It is based on Alex Shonenkov's [notebook](https://www.kaggle.com/shonenkov/inference-efficientdet), since before this I have never used EfficientDet and the documentation of the PyTorch implementation I am using is very spotty. 

## Setup

### System Detection

This is to make sure the notebook works on both my own windows workstation as well as on the Kaggle

In [ ]:
import os
system = os.name
if system == 'posix':
    kaggle = True
    windows = False
    print('running on kaggle')
elif system == 'nt':
    kaggle = False
    windows = True
    print('running on windows')
else:
    print('unknown system')

### Imports

In [ ]:
if windows:
    import os
    import numpy as np
    import pandas as pd
    from glob import glob
    import re
    import cv2
    import torch
    from torch.utils.data import DataLoader
    from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
    from effdet.efficientdet import HeadNet
    import gc
    import matplotlib.pyplot as plt
    from ensemble_boxes import *
    import albumentations as A
    from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
if kaggle:
    !pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
#     !pip install --no-deps '../input/effdetgithub/'
    !pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
if kaggle:
    import sys
    sys.path.insert(0, "../input/timm-efficientdet-pytorch")
    sys.path.insert(0, "../input/omegaconf")
    sys.path.insert(0, "../input/weightedboxesfusion")

    from ensemble_boxes import *
    import torch
    import numpy as np
    import pandas as pd
    from glob import glob
    from torch.utils.data import Dataset,DataLoader
    import albumentations as A
    from albumentations.pytorch.transforms import ToTensorV2
    import cv2
    import gc
    from matplotlib import pyplot as plt
    from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
    from effdet.efficientdet import HeadNet
    import re

### List of all test files

In [ ]:
if windows:
    path = f'../input/global-wheat-detection/train'
#     path = f'../input/global-wheat-detection/tester'
if kaggle:
    path = f'../input/global-wheat-detection/test'
jpg_paths = glob(f'{path}/*.jpg')
jpgfilenames = [re.compile(r'([\w\_\(\)]*)\.jpg').search(jpgpath).group(1) for jpgpath in jpg_paths]

### Configuration

In [ ]:
class configuration():
    scale = 1
    batch_size = 8
    if windows:
        cuda_device = 0
    if kaggle:
        cuda_device = 0
    if kaggle:
        gt = False
        visualize = False
        save = False
    if windows:
        gt = True
        visualize = True
        save = True
    picture_dims = [1024, 1024]

### DatasetRetriever

In [ ]:
# Retriever for testing data
pic_scale = {}
class DatasetRetriever():
    
    def __init__(self, image_ids, transforms = None):
        super().__init__()
        self.image_ids = image_ids
        self.scale = configuration.scale
        self.picture_dims = configuration.picture_dims
        self.transforms = transforms
    def __getitem__(self, index: int):
        if self.transforms == None:
            return self.load_item(index)
        else:
            return(self.transform_val(self.load_item(index),index, self.transforms))
    
    def load_item(self, index:int):
        filename = self.image_ids[index]
        image = f'{path}/{filename}.jpg'
        image = cv2.imread(image, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        pic_scale[filename] = image.shape
        image /= 255.0
        #resize the image
        w_image = int(image.shape[1] * self.scale)
        h_image = int(image.shape[0] * self.scale)
#         dim = (w_image, h_image)
        dim = (self.picture_dims[0],self.picture_dims[1])
        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        image = torch.tensor(image).permute(2,0,1)
        
        return image, filename
    
    def __len__(self) -> int:
        return len(self.image_ids)
    
    def transform_val(self, image_data, index, transforms):
        image, filename = image_data
    
        if transforms == 'all':
            transforms =transforms = A.Compose(
            [
                A.RandomSizedCrop(min_max_height=(400, 400), height=512, width=512, p=0.5),
                A.OneOf([
                    A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                         val_shift_limit=0.2, p=0.9),
                    A.RandomBrightnessContrast(brightness_limit=0.2, 
                                               contrast_limit=0.2, p=0.9),
                ],p=0.9),
                A.ToGray(p=0.01),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.Resize(height=512, width=512, p=1),
                A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
                ToTensorV2(p=1.0),
            ], 
            p=1.0, 
    #         bbox_params=A.BboxParams(
    #             format='pascal_voc',
    #             min_area=0, 
    #             min_visibility=0,
    #             label_fields=['labels']
    #         )
            )
        elif transforms == 'bnw':
                transforms = transforms = A.Compose([
                    A.ToGray(p=1.0),
                    ToTensorV2(p=1.0)
                    
                ], p = 1.0)
                
        elif transforms == 'solar':
                transforms = transforms = A.Compose([
                    A.RGBShift(r_shift_limit=(0.1,0.1), g_shift_limit=(0.2,0.2), b_shift_limit=(0.0,0.0), always_apply=True),
                    ToTensorV2(p=1.0)
                    
                ], p = 1.0)
                
        elif transforms == 'bright':
               transforms = transforms = A.Compose([
                   A.RandomBrightness(limit=0.2, p=1),
                   ToTensorV2(p=1.0)
                   
               ], p = 1.0)
                
        elif transforms == 'vertflip':
               transforms = transforms = A.Compose([
                   A.HueSaturationValue(hue_shift_limit=(0.0,0.0), sat_shift_limit= (0.3,0.3), 
                                         val_shift_limit=(-0.1,-0.1), p=1.0),
                    A.RandomBrightnessContrast(brightness_limit=(-0.05,-0.05), 
                                               contrast_limit=(0.3,0.3), p=1.0),
                    A.RandomGamma(gamma_limit=(75, 75), eps=None, p=1.0),
                   A.VerticalFlip(p=1.0),
                   ToTensorV2(p=1.0)
                   
               ], p = 1.0)
                
        elif transforms == 'horflip':
               transforms = transforms = A.Compose([
                   A.HueSaturationValue(hue_shift_limit=(0.0,0.0), sat_shift_limit= (0.3,0.3), 
                                         val_shift_limit=(-0.1,-0.1), p=1.0),
                    A.RandomBrightnessContrast(brightness_limit=(-0.05,-0.05), 
                                               contrast_limit=(0.3,0.3), p=1.0),
                    A.RandomGamma(gamma_limit=(75, 75), eps=None, p=1.0),
                   A.HorizontalFlip(p=1.0),
                   ToTensorV2(p=1.0)
                   
               ], p = 1.0)
                
        elif transforms == 'contrast':
               transforms = A.Compose([
                    A.HueSaturationValue(hue_shift_limit=(0.0,0.0), sat_shift_limit= (0.3,0.3), 
                                         val_shift_limit=(-0.1,-0.1), p=1.0),
                    A.RandomBrightnessContrast(brightness_limit=(-0.05,-0.05), 
                                               contrast_limit=(0.3,0.3), p=1.0),
                    A.RandomGamma(gamma_limit=(75, 75), eps=None, p=1.0),
                   ToTensorV2(p=1.0)
                   
               ], p = 1.0)
#         elif transforms == 'rot_r':
#                transforms = transforms = A.Compose([
#                    A.RandomRotate90(p=0.5, apply(factor = 3)),
#                    ToTensorV2(p=1.0)
                   
#                ], p = 1.0)        
                

        sample = transforms(**{'image': image.permute(1,2,0).cpu().numpy()#,
#               'bboxes': target['boxes'][:,[1,0,3,2]].cpu().numpy(),
#               'labels': target['labels'].cpu().numpy()
             })
        
        for i in range(10):
              if 1 == 1:
#             if len(sample['bboxes']) > 0:
                image = sample['image']
#                 target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
#                 target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]] #yxyx: be warning
#                 target['labels'] = torch.tensor(sample['labels'])
                break
        
        output = (image, filename)
        if len(output) == 2:
            return output
        else:
            return self.load_item(index)
dataset = DatasetRetriever(image_ids = jpgfilenames)

### De-augment

In [ ]:
def deaugment(image, boxes, labels, transform):
    if transform in [None, 'bnw', 'solar', 'bright','contrast']:
        return image, boxes
    else:
        
        if transform == 'vertflip':
            transforms = A.Compose(
                [A.VerticalFlip(p=1.0),
                 ToTensorV2(p=1.0)],
                p = 1.0,
            bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']))
        
        elif transform == 'horflip':
            transforms = A.Compose(
                [A.HorizontalFlip(p=1.0),
                 ToTensorV2(p=1.0)],
                p = 1.0,
            bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']))
            
            
        boxes[:,2] = boxes[:,2] + boxes[:,0]
        boxes[:,3] = boxes[:,3] + boxes[:,1]
        boxes = np.abs(np.array([boxes]))[0]
        boxes[boxes < 0] = 0
        boxes[boxes >= 1024] = 1023
        
        sample = transforms(**{'image': image.permute(1,2,0).cpu().numpy(),
                              'bboxes': boxes,
                               'labels': np.ones(boxes.shape[0])
                              })
        boxes = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
        labels = labels
        boxes[:,2] = boxes[:,2] - boxes[:,0]
        boxes[:,3] = boxes[:,3] - boxes[:,1]
        return image, boxes

            

In [ ]:

if configuration.gt == True:
    nr_files = len(jpgfilenames)
    class DatasetRetriever1():
        def __init__(self, df, image_ids):
            super().__init__()
            self.image_ids = image_ids
            self.df = df
            self.picture_dims = configuration.picture_dims
        def __getitem__(self, index:int):
            #load the image
            filename = self.image_ids[index][:-2]
            flavor = self.image_ids[index][-2:]
            filename = self.image_ids[index]
            image = f'{path}/{filename}.jpg'
            image = cv2.imread(image, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
            
            image /= 255.0
            #resize the image
            dim = (self.picture_dims[0],self.picture_dims[1])
            image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            image = torch.tensor(image).permute(2,0,1)
            if filename in labels.image_id.values:
                boxes = labels.loc[labels.image_id == filename,['x_min', 'y_min', 'x_max', 'y_max']]
                boxes = torch.tensor(boxes.values)
                boxes = boxes[:,[1,0,3,2]]
                boxes = boxes.int()
                lbls = torch.tensor(np.ones(boxes.shape[0], dtype = 'int32'))
            else:
                boxes = torch.tensor([])
                lbls = torch.tensor([])
            return image, dict(boxes = boxes, labels = lbls, image_id = torch.tensor([index])), filename
        def __len__(self):
            return len(self.image_ids)

    labels = pd.read_csv('labels.csv')
    dataset1 = DatasetRetriever1(df = labels, image_ids = jpgfilenames)
    

### Collate

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

### Data Loader 

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=configuration.batch_size,
                                          pin_memory = False,
                                          drop_last = False,
                                          shuffle = False,
                                          collate_fn = collate_fn)

### Load the Network

In [ ]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config = config, pretrained_backbone = False)
    config.num_classes = 1
    config.image_size = 1024 * configuration.scale
    net.class_net = HeadNet(config, num_outputs = config.num_classes,
                            norm_kwargs = dict(eps = .001, momentum = .01))
    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    
    
    del checkpoint
    gc.collect()
    if windows:
        net = DetBenchPredict(net, config)
    if kaggle:
        net = DetBenchEval(net, config)
    return net.cuda(configuration.cuda_device)


In [ ]:
if windows:
#     chkpntpth = sorted(glob(f'../../train/working/effdet-experimental/best-checkpoint-*epoch.bin'))[-1]
    chkpntpth =  f'../../train/working/effdet-experimental/best-checkpoint-026epoch.bin'
    chkpntpth2 = f'../../train/working/effdet-experimental/last-checkpoint.bin'
    chkpntpth3 = f'../../train/working/effdet-experimental/last-checkpoint-kaggle.bin'
    chkpntpth4 = f'../../train/working/effdet-experimental/best-checkpoint-030epoch.bin'
if kaggle:
    chkpntpth = f'../input/effdetweights/best-checkpoint-026epoch.bin'
    chkpntpth2 = f'../input/effdetweights/last-checkpoint.bin'
    chkpntpth3 = f'../input/effdetweights/last-checkpoint-kaggle.bin'
    chkpntpth4 = f'../input/effdetweights/best-checkpoint-030epoch.bin'
    
net = load_net(chkpntpth)
net2 = load_net(chkpntpth2)
net3 = load_net(chkpntpth3)
net4 = load_net(chkpntpth4)


### Make Predictions

In [ ]:
def make_predictions(images, image_ids, model, transforms, score_threshold_orig):
    prediction_list = []
    images = torch.stack(images).cuda(configuration.cuda_device).float()
    #, 'bad_conf':[], 'badboxes':[]}
    
    batch_size = images.shape[0]
    images = images.cuda(configuration.cuda_device).float()
    target_res = {}
    target_res['img_scale'] = torch.tensor([1.0] * batch_size, dtype = torch.float32).cuda(configuration.cuda_device)
    target_res['img_size'] = torch.tensor([images[0].shape[-2:]] * batch_size, dtype=torch.float32).cuda(configuration.cuda_device)
    with torch.no_grad():
        if windows:
            det = model(images, target_res['img_scale'], target_res['img_size'])
        if kaggle:
            det = model(images, torch.tensor([1]*images.shape[0]).float().cuda())
            
        for i in range(images.shape[0]):
            predictions = {'image_id':[],'conf': [],'boxes':[]}
            # the first four columns are the boundaries of the boxes and the last 2 are the 
            score_threshold = score_threshold_orig
            boxes = det[i].detach().cpu().numpy()[:,:4]
            nonull = boxes[:,2] > 10
            boxes = boxes[nonull]
            boxes = deaugment(images[i],boxes,np.ones(boxes.shape[0]), transforms)[1]
            scores = det[i].detach().cpu().numpy()[:,4]
            scores = scores[nonull]
            if boxes.shape[0] < 5:
                score_threshold = 0.6
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            scores = scores[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            #predictions.append({'boxes': boxes[indexes],
             #                 'scores': scores[indexes]})
            predictions['boxes'] = boxes
            predictions['conf'] = scores
            predictions['image_id'] = image_ids[i]
#             predictions['bad_conf'].append()
            prediction_list.append(predictions)
    return prediction_list
#     with torch.no_grad():
        

### Ensemble Boxes

In [ ]:
def apply_wbs(images, image_ids,models, transform_list,
              score_threshold_orig = 0.2, skip_box_thr = 0.001):
    prediction_list = []
    # make image_index_list
    index_list = []
    for image_id in image_ids:
#         image_index = DatasetRetriever(image_ids = jpgfilenames, transforms = None).__getitem__(3)[1]
        image_index = jpgfilenames.index(image_id)
        index_list.append(image_index)
    for model in models:
        for transform in transform_list:
            image_list = []
            for index in index_list:
                image = DatasetRetriever(image_ids = jpgfilenames,
                                         transforms = transform).__getitem__(index)
                image_list.append(image[0])
            images = image_list
            prediction_list.append(make_predictions(images, image_ids,
                                                    score_threshold_orig = score_threshold_orig,
                                                    model = model,transforms = transform))
    predicts = []
    prediction_df = pd.DataFrame(columns = ['image_id','conf','boxes'])
    for prediction in prediction_list:
        prediction_df = pd.concat([prediction_df,pd.DataFrame(prediction)], axis = 0)
    nr_rows = prediction_df.shape[0]
    pic_size = configuration.picture_dims[0]
    prediction_df['normed_boxes'] = prediction_df['boxes'] / pic_size
    for unique_id in prediction_df.image_id.unique():
        boxes = []
        scores = []
        boxes_list = []
        scores_list = []
        labels_list = []
        small_df = prediction_df[prediction_df.image_id == unique_id].copy()
        for i in range(small_df.shape[0]):
            boxes_list.append(small_df.iloc[i].normed_boxes)
            scores_list.append(small_df.iloc[i].conf)
            labels_list.append([1]*small_df.iloc[i].conf.shape[0])
        weights = [1]*len(models)*len(transform_list)
        iou_thr = 0.45
#         print(boxes_list)
#         max_len = 0
#         for array in scores_list:
#             if len(array) > max_len:
#                 max_len = len(array)
#             if (max_len == 0):
#                 predicts.append({'image_id' : unique_id, 'conf' : scores_list[0].astype(np.float32), 'boxes' : boxes_list[0].astype(np.float32)})
        
        if 1==0:
            print('nothing')


        else:
            if len(models) == 1:
                print('only one model')
                boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,
                                                          weights = None, iou_thr = iou_thr,
                                                         skip_box_thr = skip_box_thr)
            else:
                boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,
                                                          weights = weights, iou_thr = iou_thr,
                                                         skip_box_thr=skip_box_thr,conf_type='avg')

        thresh = 0.37
        
#         print(thresh)
    
        #             print(thresh)
        indexes = np.where(scores > thresh)[0]
        
        boxes1 = boxes[indexes]
        scores1 = scores[indexes]
        if boxes.shape[0] < 5:
            thresh = 0.5
            indexes1 = np.where(scores > thresh)[0]
            boxes1 = boxes1[indexes1]
            scores1 = scores1[indexes1]
        #             print(np.array(boxes,dtype = int))
        
        prediction_dict = {'image_id' : unique_id, 'conf': scores1.astype(np.float32), 'boxes': boxes1.astype(np.float32)*pic_size}
        predicts.append(prediction_dict)
    return predicts

### Visualize Predictions

In [ ]:
# transform_list = [None,'vertflip','horflip']#'solar','bright','horflip','bnw']
transform_list = ['contrast','vertflip','horflip']
model_list = [net2,net3]

In [ ]:
if configuration.visualize == True:
    
    dataset2 = DatasetRetriever(image_ids = jpgfilenames, transforms = None)
    data_loader2 = torch.utils.data.DataLoader(dataset2,
                                          batch_size=configuration.batch_size,
                                          pin_memory = False,
                                          drop_last = False,
                                          shuffle = False,
                                          collate_fn = collate_fn)
    
    m = 0
    new_imagelist = []
    inf_folder = f'./inference'
    if configuration.save & (not os.path.exists(inf_folder)):
                os.makedirs(inf_folder)
    for j, (images, image_ids) in enumerate(data_loader2):
        predictions = apply_wbs(images,image_ids, models = model_list,
                                transform_list = transform_list) 
#         predictions = make_predictions(images, image_ids)
        df = pd.DataFrame(predictions)
        for i in range(len(image_ids)):
            ## plotting the inference
            image_id = image_ids[i]
            new_imagelist.append(image_id)
            fig, ax = plt.subplots(1, 1, figsize=(16, 16))
            bx1 = df.iloc[i].boxes
            conf1 = df.iloc[i].conf
            sample = images[i].permute(1,2,0).cpu().numpy()
            k = 0
            for box in bx1:
                cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
                plt.text(box[2],box[1],f'{conf1[k]:.3f}', color = 'red', fontsize = 18)
                k+=1
            #Plotting the Ground Truth
            if configuration.gt == True:
                image1, target1, image_id1 = dataset1[m]
                boxes1 = target1['boxes'].cpu().numpy().astype(np.int32)
                for box1 in boxes1:
                    cv2.rectangle(sample, (box1[1], box1[0]), (box1[3],  box1[2]), (0, 1, 0), 2)

            #Showing and saving the picture
            ax.set_axis_off()
            ax.imshow(sample)
            if windows & configuration.save:
                plt.savefig(f'{inf_folder}/{image_ids[i]}.jpeg', transparent=True, bbox_inches = 'tight',
                            facecolor = 'k',pad_inches = 0)
            plt.show()
            m+=1

In [ ]:
submission = pd.DataFrame(columns = ['image_id', 'PredictionString'])
cols = ['conf', 'px_min', 'py_min', 'px_max', 'py_max']

for j, (images, image_ids) in enumerate(data_loader):
    predictions = apply_wbs(images, image_ids, models = model_list,transform_list = [None,'vertflip','solar','bright','horflip']) 
    df = pd.DataFrame(predictions)
    for i in range(len(image_ids)):
        row = df.iloc[i]
        filename = row.image_id
        dataset = pd.DataFrame(columns = cols)
        if len(row.conf) > 0:
            for k in range(len(row.conf)):
                row_df = pd.DataFrame([[row.conf[k],row.boxes[k][0],row.boxes[k][1],row.boxes[k][2],row.boxes[k][3]]], 
                                      columns = cols)
                dataset = pd.concat([dataset, row_df], axis = 0, ignore_index = True)
            dataset['width'] = configuration.picture_dims[0]
            dataset['height'] = configuration.picture_dims[1]
            scale_x = pic_scale[filename][1]/1024
            scale_y = pic_scale[filename][0]/1024
            dataset['px_box'] = dataset['px_max'] - dataset['px_min']
            dataset['py_box'] = dataset['py_max'] - dataset['py_min']
            dataset[['px_min', 'py_min', 'px_box', 'py_box']] = dataset[['px_min', 'py_min', 'px_box', 'py_box']].round().astype('int')
            dataset.loc[dataset.width < dataset.px_min,'px_min'] = dataset.width -1
            dataset.loc[dataset.height < dataset.py_min,'py_min'] = dataset.height -1
            dataset.loc[1 > dataset.px_box,'px_box'] = 1
            dataset.loc[1 > dataset.py_box,'py_box'] = 1
            dataset.loc[0 > dataset.px_min,'px_min'] = 0
            dataset.loc[0 > dataset.py_min,'py_min'] = 0
            dataset.loc[dataset['px_min'] + dataset['px_box'] > dataset['width'], 'px_box'] = dataset['width'] - dataset['px_min'] - 1
            dataset.loc[dataset['py_min'] + dataset['py_box'] > dataset['height'], 'py_box'] = dataset['height'] - dataset['py_min'] - 1
            dataset[['px_min','px_box']] = (dataset[['px_min','px_box']] * scale_x).astype(int)
            dataset[['py_min','py_box']] = (dataset[['py_min','py_box']] * scale_y).astype(int)
            submission_cols = ['conf', 'px_min', 'py_min', 'px_box', 'py_box']
            shape = dataset[submission_cols].shape
            row1 = dataset[submission_cols].values.reshape(1,shape[0]*shape[1])
            str_array = np.array2string(row1[0], formatter = {'float_kind': lambda x: '%.5f' % x}, max_line_width = 9e3)
            str_array = re.compile(r'\[([\d\s\.]+)\]').search(str_array).group(1)
            vals = pd.DataFrame([[filename, str_array]], columns =['image_id', 'PredictionString'])
        else:
            vals = pd.DataFrame([[filename, '']], columns =['image_id', 'PredictionString'])
        submission = pd.concat([submission,vals], axis = 0, ignore_index = True)
        submission.PredictionString = submission.PredictionString.str.replace(r'(\.0+)', '').str.replace('\s+',' ')

submission.to_csv('submission.csv', index = False, sep = ',')
submission